In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import csv
import pandas as pd
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.119.52.50:8470


W0823 03:15:26.347785 139705724729216 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 16433370519026702276),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11508507013745903813),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13961404652946566935),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10438339571033082959),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3458348512851077469),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 12386988350920086961),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10791587568688378005),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 1420336064106645730),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1559729432878512747),
 _DeviceAttributes(/job:tpu_

In [2]:
import sys
from os import listdir
!rm -rf bert_finetune
!test -d bert_finetune || git clone https://github.com/junhahyung/bert_finetune bert_finetune
if not 'bert_finetune' in sys.path:
  sys.path += ['bert_finetune']
print(listdir("bert_finetune"))
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_finetune'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 74 (delta 29), reused 39 (delta 11), pack-reused 0
Unpacking objects: 100% (74/74), done.


W0823 03:15:31.516349 139705724729216 deprecation_wrapper.py:119] From bert_finetune/optimization.py:88: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



['run_pretraining.py', 'optimization_test.py', 'modeling_test.py', 'multilingual.md', 'LICENSE', 'extract_features.py', 'run_classifier_with_tfhub.py', 'README.md', '__init__.py', 'predicting_movie_reviews_with_bert_on_tf_hub.ipynb', 'run_squad.py', 'tokenization_test.py', '__init__.pyc', 'modeling.py', 'create_pretraining_data.py', 'run_classifier.py', '.gitignore', 'modeling.pyc', 'requirements.txt', 'sample_text.txt', 'tokenization.py', 'CONTRIBUTING.md', 'optimization.py', '.git']


In [3]:
%cd /content
!test -d khaiii || git clone https://github.com/kakao/khaiii.git
%cd /content/khaiii
!ls
!test -d build || mkdir build
%cd /content/khaiii/build
!cmake ..
!make all
!make resource

/content
Cloning into 'khaiii'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 847 (delta 92), reused 158 (delta 53), pack-reused 601
Receiving objects: 100% (847/847), 30.29 MiB | 28.56 MiB/s, done.
Resolving deltas: 100% (292/292), done.
/content/khaiii
cmake		 docker   munjong    requirements.txt  train
CMakeLists.txt	 include  NOTICE.md  rsc
CONTRIBUTING.md  LICENSE  README.md  src
/content/khaiii/build
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI inf

In [4]:
%cd /content/khaiii/build
!ctest
!make package_python
%cd /content/khaiii/build/package_python
!pip install .
%cd /content/khaiii
sys.path.append('/content/khaiii')
from khaiii import KhaiiiApi
api = KhaiiiApi()
for word in api.analyze('안녕, 세상.'):
    print(word)

/content/khaiii/build
Test project /content/khaiii/build
    Start 1: test_khaiii
1/1 Test #1: test_khaiii ......................   Passed    0.02 sec

100% tests passed, 0 tests failed out of 1

Total Test time (real) =   0.04 sec
Scanning dependencies of target package_python
Run CPack packaging tool for source...
CPack: Create package using ZIP
CPack: Install projects
CPack: - Install directory: /content/khaiii
CPack: Create package
CPack: - package: /content/khaiii/build/khaiii-0.4.zip generated.
Built target package_python
/content/khaiii/build/package_python
Processing /content/khaiii/build/package_python
  Created wheel for khaiii: filename=khaiii-0.4-cp36-none-any.whl size=22891189 sha256=24d2124bbb558c3ef54473b70f7202a4db631a0673d426ab42fd326bdd05fc86
  Stored in directory: /tmp/pip-ephem-wheel-cache-mo6j1fib/wheels/b0/03/83/fb129110ddf28d31298b0731cd182d7d0f094935ae9bf8d4bc
Successfully built khaiii
/content/khaiii
안녕,	안녕/IC + ,/SP
세상.	세상/NNG + ./SF


In [5]:
TASK = 'korean_sa'
assert TASK in ('MRPC', 'CoLA', 'korean_sa'), 'Undefined task'

BUCKET = 'mbertfinetune' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
TASK_DATA_DIR = 'gs://{}/data/{}'.format(BUCKET, TASK)
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
print(tf.gfile.ListDirectory(TASK_DATA_DIR))
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

"""
tdir = os.path.join(TASK_DATA_DIR, "korean_train.csv")
with tf.gfile.Open(tdir, "r") as f:
  reader = pd.read_csv(tdir) 
  lines = []
  for line in reader:
    lines.append(line)
print(lines[18824])
"""
VOCAB_DIR = 'gs://{}/{}/vocab.txt'.format(BUCKET, BERT_MODEL)
print(VOCAB_DIR)

***** Task data directory: gs://mbertfinetune/data/korean_sa *****
['korean_dev.csv', 'korean_train.csv']
***** Model output directory: gs://mbertfinetune/bert-tfhub/models/korean_sa *****
gs://mbertfinetune/multi_cased_L-12_H-768_A-12/vocab.txt


In [6]:
import collections
def load_vocab(vocab_file):
  vocab = collections.OrderedDict()
  index = 0
  with tf.gfile.GFile(vocab_file, "r") as reader:
    while True:
      token = reader.readline()
      if not token:
        break
      token = token.strip()
      vocab[token] = index
      index += 1
  return vocab

class K_tokenizer:
  def __init__(self):
    self.api = KhaiiiApi()
    self.vocab = load_vocab(VOCAB_DIR)
  def tokenize(self, text):
    words = []
    for word in api.analyze(text):
      word = str(word).split('\t')
      word = word[1]
      word = word.split(' + ')
      cnt = 0
      for sub in word:
        sub = sub.split('/')[0]
        if cnt == 0:
          pass
        elif sub == '.' or sub == '?' or sub == '!':
          pass
        else:
          sub = '##' + sub
        words.append(sub)
        cnt += 1
    return words
  def convert_tokens_to_ids(self, tokens):
    output = []
    for item in tokens:
      if item in self.vocab:
        output.append(self.vocab[item])
      else:
        ind = len(self.vocab)
        self.vocab[item] = ind  
        output.append(self.vocab[item])
    return output
tokenizer = K_tokenizer()
tokenizer.tokenize("한국어는 잘 안되는 것 같아요.")


['한국어', '##는', '잘', '안', '##되', '##는', '것', '같', '##아요', '.']

In [0]:
#tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
#tokenizer.tokenize("한국어는 잘 안되는 것 같아요.")

In [8]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 5e-5
NUM_TRAIN_EPOCHS = 3
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
layer_wise_lr = (True, 0.3)
# layer_wise_lr = None 

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "korean_sa": run_classifier.KsaProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

processor.get_train_examples(TASK_DATA_DIR)
# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000 

def get_run_config(output_dir):  
  return tf.contrib.tpu.RunConfig(    
      cluster=tpu_cluster_resolver,    
      model_dir=output_dir,    
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,    
      tpu_config=tf.contrib.tpu.TPUConfig(        
          iterations_per_loop=ITERATIONS_PER_LOOP,        
          num_shards=NUM_TPU_CORES,        
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

W0823 03:28:35.008950 139705724729216 deprecation_wrapper.py:119] From bert_finetune/run_classifier.py:382: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [9]:
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR 

model_fn = run_classifier_with_tfhub.model_fn_builder(  
    num_labels=len(label_list),  
    learning_rate=LEARNING_RATE,  
    num_train_steps=num_train_steps,  
    num_warmup_steps=num_warmup_steps,  
    use_tpu=True,  
    bert_hub_module_handle=BERT_MODEL_HUB,
    layer_wise_lr=layer_wise_lr
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

W0823 03:28:36.872664 139705724729216 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f0f7bd3ba60>) includes params argument, but params are not passed to Estimator.


In [0]:
# Train the model
def model_train(estimator):
  print('Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [11]:
model_train(estimator_from_tfhub)

W0823 03:28:36.904911 139705724729216 deprecation_wrapper.py:119] From bert_finetune/run_classifier.py:837: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...
***** Started training at 2019-08-23 03:28:46.110111 *****
  Num examples = 8961
  Batch size = 32


W0823 03:28:46.505763 139705724729216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
E0823 03:29:43.707326 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:29:43.709222 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:29:43.712125 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in t

***** Finished training at 2019-08-23 03:34:50.777289 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [13]:
model_eval(estimator_from_tfhub)

***** Started evaluation at 2019-08-23 03:34:53.940759 *****
  Num examples = 2194
  Batch size = 8


E0823 03:35:05.442262 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:35:05.448425 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:35:05.452384 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:35:05.454158 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E0823 03:35:05.465035 139705724729216 tpu.py:376] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

***** Finished evaluation at 2019-08-23 03:36:05.789547 *****
***** Eval results *****
  eval_accuracy = 0.5615876
  eval_loss = 1.155905
  global_step = 840
  loss = 1.1857085
